In [2]:
!pip install albumentations timm pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 18.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylin

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Train

In [4]:
import warnings
warnings.filterwarnings('ignore')

import os, zipfile
import pandas as pd
import numpy as np
import albumentations as A
import torch
import cv2
import timm
import random
import shutil

from albumentations.pytorch import ToTensorV2
from PIL import Image
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader, random_split
from tqdm.auto import notebook_tqdm

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(42)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [9]:
def dataset_extract(file_name) :
    with zipfile.ZipFile(file_name, 'r') as zip_ref :
        file_list = zip_ref.namelist()
        if os.path.exists(f'{file_name[-8:-4]}/') :
            print(f'데이터셋 폴더가 이미 존재합니다.')
            return

        else :
            for f in notebook_tqdm(file_list, desc='Extracting', unit='files') :
                zip_ref.extract(member=f, path=f'/content/')

dataset_extract('/content/drive/MyDrive/프로젝트/조류 이미지 분류/open.zip')

Extracting:   0%|          | 0/38460 [00:00<?, ?files/s]

In [10]:
class CustomDataset(Dataset):
    def __init__(self, csv_file, root_dir, mode='train', transform=None):
        self.df = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform
        self.mode = mode

        if mode == 'train':
            self.label_encoder = LabelEncoder()
            self.df['label'] = self.label_encoder.fit_transform(self.df['label'])

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = os.path.join(self.root_dir, self.df.iloc[idx, 0])
        image = Image.open(img_name)
        image = np.array(image)

        if self.transform:
            augmented = self.transform(image=image)
            image = augmented['image']

        label = self.df.iloc[idx, 2] if self.mode == 'train' else -1

        return image, label

In [11]:
mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)

train_transform = A.Compose([
    A.Resize(196, 196, interpolation=cv2.INTER_CUBIC),
    # A.HorizontalFlip(p=0.5),
    # A.RandomBrightnessContrast(p=0.2),
    A.Normalize(mean=mean, std=std),
    ToTensorV2()
])

In [12]:
train_dataset = CustomDataset(csv_file="train.csv", root_dir="", mode='train', transform=train_transform)

total_train_samples = len(train_dataset)
val_size = int(0.1 * total_train_samples)
train_size = total_train_samples - val_size

train_subset, val_subset = random_split(train_dataset, [train_size, val_size])

train_loader = DataLoader(train_subset, batch_size=32, shuffle=True, num_workers=12)
val_loader = DataLoader(val_subset, batch_size=32, shuffle=False, num_workers=12) # num_workers: 데이터를 로드하는 데 사용할 프로세스(worker)의 수

In [13]:
model = timm.create_model("timm/eva_large_patch14_196.in22k_ft_in22k_in1k", pretrained=True, num_classes=25)
model.to(device)
model = torch.nn.DataParallel(model)

optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
criterion = torch.nn.CrossEntropyLoss()
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=1, verbose=True, min_lr=1e-6)

In [14]:
def accuracy(pred, true):
  _, preds = torch.max(pred, dim=1)
  return torch.tensor(torch.sum(preds == true).item() / len(preds))

def train(model, dataloader, criterion, optimizer, device):
    model.train()

    train_loss = 0.0
    train_acc = 0.0

    tqdm_bar = notebook_tqdm(dataloader, desc='Training')
    for batch_idx, (images, labels) in enumerate(tqdm_bar):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        train_acc += accuracy(outputs, labels).item()

        avg_loss = train_loss / (batch_idx + 1)
        avg_acc = train_acc / (batch_idx + 1)

        tqdm_bar.set_postfix(
            {'loss': f'{avg_loss:.5f}',
            'accuracy': f'{avg_acc:.5f}'}
        )
    return train_loss / len(dataloader), train_acc / len(dataloader)

def validate(model, dataloader, criterion, device):
    model.eval()

    val_loss = 0.0
    val_acc = 0.0

    tqdm_bar = notebook_tqdm(dataloader, desc='Training')
    for batch_idx, (images, labels) in enumerate(tqdm_bar):
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        val_loss += loss.item()
        val_acc += accuracy(outputs, labels).item()

        avg_loss = val_loss / (batch_idx + 1)
        avg_acc = val_acc / (batch_idx + 1)

        tqdm_bar.set_postfix(
            {'loss': f'{avg_loss:.5f}',
            'accuracy': f'{avg_acc:.5f}'}
        )
    return val_loss / len(dataloader), val_acc / len(dataloader)

In [15]:
best_weight_path = "./weights/eva-large-196-best.pth"
current_weight_path = "./weights/eva-large-196-current.pth"

patience = 3
num_epochs = 10

if os.path.exists(current_weight_path):
    checkpoint = torch.load(current_weight_path)
    best_checkpoint = torch.load(best_weight_path)

    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

    best_val_loss = best_checkpoint['best_val_loss']
    early_stopping_counter = checkpoint['early_stopping_counter']
    start_epoch = checkpoint['epoch'] + 1
    print('Loaded model from last checkpoint')
    print("Last best validation loss: ", best_val_loss)
    print("Continuing from epoch: ", start_epoch)
    print("Early stopping counter: ", early_stopping_counter)
else:
    best_val_loss = float('inf')
    early_stopping_counter = 0
    start_epoch = 0

In [16]:
for epoch in range(start_epoch, num_epochs):
    train_loss, train_acc = train(model, train_loader, criterion, optimizer, device)
    val_loss, val_acc = validate(model, val_loader, criterion, device)
    current_lr = optimizer.param_groups[0]['lr']
    print(f"Epoch {epoch+1}/{num_epochs}, LR: {current_lr}")
    print(f"Train Loss: {train_loss:.5f}, Train Accuracy: {train_acc:.5f}")
    print(f"Val Loss: {val_loss:.5f}, Val Accuracy: {val_acc:.5f}")

    scheduler.step(val_loss)

    current_checkpoint = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'val_loss': val_loss,
        'early_stopping_counter': early_stopping_counter
    }

    torch.save(current_checkpoint, current_weight_path)
    print('Model saved')

    ''' Save the weights with the best validation loss '''
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_checkpoint = {
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'best_val_loss': best_val_loss,
            'early_stopping_counter': early_stopping_counter
        }

        torch.save(best_checkpoint, best_weight_path)
        early_stopping_counter = 0
        print('Best model saved')
    else:
        early_stopping_counter += 1
        if early_stopping_counter >= patience:
            print(f'Early stopping at epoch: {epoch+1}')
            break
print('Training finished')

Training:   0%|          | 0/446 [00:00<?, ?it/s]

Training:   0%|          | 0/50 [00:00<?, ?it/s]

Epoch 1/10, LR: 1e-05
Train Loss: 0.39168, Train Accuracy: 0.90205
Val Loss: 0.10402, Val Accuracy: 0.97688
Model saved
Best model saved


Training:   0%|          | 0/446 [00:00<?, ?it/s]

Training:   0%|          | 0/50 [00:00<?, ?it/s]

...

Training:   0%|          | 0/446 [00:00<?, ?it/s]

Training:   0%|          | 0/50 [00:00<?, ?it/s]

Epoch 6/10, LR: 1e-05
Train Loss: 0.00076, Train Accuracy: 1.00000
Val Loss: 0.08948, Val Accuracy: 0.97562
Model saved


Training:   0%|          | 0/446 [00:00<?, ?it/s]

KeyboardInterrupt: 

### Inference

In [ ]:
import torch
import numpy as np
import pandas as pd
import cv2
import albumentations as A
import timm
import os

from albumentations.pytorch import ToTensorV2
from PIL import Image
from tqdm.auto import notebook_tqdm
from torch.utils.data import DataLoader, Dataset
from sklearn.preprocessing import LabelEncoder

In [ ]:
model_path = "weights/eva-large-196-best.pth"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, csv_file, root_dir, mode='train', transform=None):
        self.df = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform
        self.mode = mode

        if mode == 'train':
            self.label_encoder = LabelEncoder()
            self.df['label'] = self.label_encoder.fit_transform(self.df['label'])

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = os.path.join(self.root_dir, self.df.iloc[idx, 1])
        image = Image.open(img_name)
        image = np.array(image)

        if self.transform:
            augmented = self.transform(image=image)
            image = augmented['image']

        label = self.df.iloc[idx, 2] if self.mode == 'train' else -1

        return image, label

In [ ]:
mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)

test_transform = A.Compose([
    A.Resize(196, 196, interpolation=cv2.INTER_CUBIC),
    A.Normalize(mean=mean, std=std),
    ToTensorV2()
])

In [ ]:
test_dataset = CustomDataset(csv_file="test.csv", root_dir="", mode='test', transform=test_transform)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=4)

model = timm.create_model("timm/eva_large_patch14_196.in22k_ft_in22k_in1k", pretrained=False, num_classes=25)
model.to(device)
model = torch.nn.DataParallel(model)

In [ ]:
checkpoint = torch.load(model_path, map_location=device)
model.load_state_dict(checkpoint['model_state_dict'])

def test(model, dataloder):
    predictions = []
    model.eval()

    tqdm_bar = notebook_tqdm(dataloder, desc="Predicting")
    for images, _ in tqdm_bar:
        images = images.to(device)

        with torch.no_grad():
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            predictions.extend(preds.cpu().numpy())
    return predictions

In [ ]:
test_predictions = test(model, test_loader)

le = LabelEncoder()

train_df = pd.read_csv("train.csv")
le.fit(train_df["label"])

final_prediction = le.inverse_transform(test_predictions)

Predicting:   0%|          | 0/6786 [00:00<?, ?it/s]

In [ ]:
submission_df = pd.read_csv("sample_submission.csv")
submission_df['label'] = final_prediction
submission_df.to_csv("/content/drive/MyDrive/프로젝트/조류 이미지 분류/sub/eva_large_patch14_196.csv", index=False)